In [1]:
# Import necessary libraries
import os
import pandas as pd

# Function to transform a CSV file
def transform_csv(input_path):
    # Load the CSV file
    ogscn_data = pd.read_csv(input_path)

    # Drop columns with 'Unnamed' in the column name and columns where all values are NaN
    ogscn_data.drop(columns=ogscn_data.columns[ogscn_data.columns.str.startswith('Unnamed')], inplace=True)
    ogscn_data.dropna(axis=1, how='all', inplace=True)

    # Extract COICOP and CDID identifier codes and drop the rows
    coicop_codes, cdid_codes = ogscn_data.iloc[0].to_dict(), ogscn_data.iloc[1].to_dict()
    ogscn_data.drop([0, 1], inplace=True)
    ogscn_data.reset_index(drop=True, inplace=True)

    # Melt the dataframe
    melted_data = pd.melt(ogscn_data, id_vars=['Time period and codes'], value_vars=ogscn_data.columns[1:])
    melted_data['NamedCost'] = melted_data['variable']
    melted_data['COICOP'] = melted_data['variable'].map(coicop_codes)
    melted_data['CDID'] = melted_data['variable'].map(cdid_codes)
    melted_data["Source"] = os.path.splitext(os.path.basename(input_path))[0]

    # Transform columns
    melted_data['Year'] = melted_data['Time period and codes'].str.split(' ').str[0]
    melted_data['Quarter'] = melted_data['Time period and codes'].str.split(' ').str[1]
    melted_data.drop(columns=['variable', 'Time period and codes'], inplace=True)

    # Rearrange the columns
    column_order = ['Year', 'Quarter', 'NamedCost', 'value', 'COICOP', 'CDID', 'Source']
    melted_data = melted_data[column_order]
    melted_data.columns = ['Year', 'Quarter', 'NamedCost', 'Values', 'COICOP', 'CDID', 'Source']

    return melted_data

# Define input and output folders
input_folder = 'C:/Users/l_cla/Documents/000_Project/000_000_datastored/100_raw/'
output_folder = 'C:/Users/l_cla/Documents/000_Project/000_000_datastored/200_structured/'

# List of file names to process
file_names = ["0GSCN.csv", "01CN.csv", "02CN.csv", "03CN.csv", "04CN.csv", "05CN.csv", "06CN.csv", 
              "07CN.csv", "08CN.csv", "10CN.csv", "11CN.csv", "12CN.csv",
              "DGCN.csv", "NDGCN.csv", "OCN.csv", "SDGCN.csv", "SERCN.csv", "TGCN.csv",
              "TOURCN.csv"]  # Add your file names here

# List to store transformed DataFrames
dfs = []

# Process each file
for file_name in file_names:
    input_path = os.path.join(input_folder, file_name)
    df_transformed = transform_csv(input_path)
    dfs.append(df_transformed)

# Concatenate all DataFrames
merged_df = pd.concat(dfs, ignore_index=True)

# Filter out rows with missing 'Year' or 'Quarter'
merged_df = merged_df.dropna(subset=['Year', 'Quarter'])
merged_df = merged_df[merged_df['Year'] != '']
merged_df = merged_df[merged_df['Quarter'] != '']

# Save the merged data
#merged_output_path = os.path.join(output_folder, 'merged_transformed_data.csv')
#merged_df.to_csv(merged_output_path, index=False)

# Display the merged DataFrame and a completion message
print(merged_df)
print("Complete")


       Year Quarter                      NamedCost   Values COICOP  CDID  \
0      1997      Q1              Total expenditure  144,887   NAT0  ABPB   
1      1997      Q2              Total expenditure  150,474   NAT0  ABPB   
2      1997      Q3              Total expenditure  155,809   NAT0  ABPB   
3      1997      Q4              Total expenditure  159,193   NAT0  ABPB   
4      1998      Q1              Total expenditure  153,347   NAT0  ABPB   
...     ...     ...                            ...      ...    ...   ...   
43970  2021      Q4  UK tourist expenditure abroad    9,723  TOUR2  ABTA   
43971  2022      Q1  UK tourist expenditure abroad   10,062  TOUR2  ABTA   
43972  2022      Q2  UK tourist expenditure abroad   18,447  TOUR2  ABTA   
43973  2022      Q3  UK tourist expenditure abroad   24,854  TOUR2  ABTA   
43974  2022      Q4  UK tourist expenditure abroad   15,623  TOUR2  ABTA   

       Source  
0       0GSCN  
1       0GSCN  
2       0GSCN  
3       0GSCN  
4      

In [2]:
# Check if 'CDID' column exists
if 'CDID' in merged_df.columns:
    # Create the 'Websource' column by appending the 'CDID' to the base URL
    merged_df['Websource'] = "https://www.ons.gov.uk/search?q=" + merged_df['CDID']
else:
    error_message = "'CDID' column does not exist in the provided file."

# Check the first few rows of the modified DataFrame
#merged_df.head() if 'CDID' in merged_df.columns else error_message

# Display the modified DataFrame
print(merged_df)

print("Complete")


       Year Quarter                      NamedCost   Values COICOP  CDID  \
0      1997      Q1              Total expenditure  144,887   NAT0  ABPB   
1      1997      Q2              Total expenditure  150,474   NAT0  ABPB   
2      1997      Q3              Total expenditure  155,809   NAT0  ABPB   
3      1997      Q4              Total expenditure  159,193   NAT0  ABPB   
4      1998      Q1              Total expenditure  153,347   NAT0  ABPB   
...     ...     ...                            ...      ...    ...   ...   
43970  2021      Q4  UK tourist expenditure abroad    9,723  TOUR2  ABTA   
43971  2022      Q1  UK tourist expenditure abroad   10,062  TOUR2  ABTA   
43972  2022      Q2  UK tourist expenditure abroad   18,447  TOUR2  ABTA   
43973  2022      Q3  UK tourist expenditure abroad   24,854  TOUR2  ABTA   
43974  2022      Q4  UK tourist expenditure abroad   15,623  TOUR2  ABTA   

       Source                             Websource  
0       0GSCN  https://www.ons.go

In [3]:
# Chain transformations for the 'Values' column
# Remove spaces and commas from the 'Values' column
merged_df['Values'] = merged_df['Values'].str.replace(' ', '', regex=False).str.replace(',', '')

# Exclude rows with '[x]' in the 'Values' column
merged_df = merged_df[merged_df['Values'] != '[x]']

# Convert the 'Values' column to float and multiply by a million
merged_df['Values'] = merged_df['Values'].astype(float) * 1e6

# Display the first few rows of the transformed DataFrame
print(merged_df)
print("Complete")


       Year Quarter                      NamedCost        Values COICOP  CDID  \
0      1997      Q1              Total expenditure  1.448870e+11   NAT0  ABPB   
1      1997      Q2              Total expenditure  1.504740e+11   NAT0  ABPB   
2      1997      Q3              Total expenditure  1.558090e+11   NAT0  ABPB   
3      1997      Q4              Total expenditure  1.591930e+11   NAT0  ABPB   
4      1998      Q1              Total expenditure  1.533470e+11   NAT0  ABPB   
...     ...     ...                            ...           ...    ...   ...   
43970  2021      Q4  UK tourist expenditure abroad  9.723000e+09  TOUR2  ABTA   
43971  2022      Q1  UK tourist expenditure abroad  1.006200e+10  TOUR2  ABTA   
43972  2022      Q2  UK tourist expenditure abroad  1.844700e+10  TOUR2  ABTA   
43973  2022      Q3  UK tourist expenditure abroad  2.485400e+10  TOUR2  ABTA   
43974  2022      Q4  UK tourist expenditure abroad  1.562300e+10  TOUR2  ABTA   

       Source              

In [4]:
# Chain transformations for the 'NamedCost' column
merged_df['NamedCost'] = (
    merged_df['NamedCost']
    .str.replace(',', '', regex=False)            # Remove commas
    .str.replace(' +', '', regex=True)            # Remove multiple spaces
    .str.replace(' ', '_', regex=False)           # Replace spaces with underscores
    .str.replace(r'\[([^\]]+)\]', r'_\1', regex=True)  # Replace [text] with _text
    .str.replace('__', '_', regex=False)         # Replace consecutive underscores with a single underscore
    .str.replace('.', '', regex=False)           # Remove periods
    .str.replace('-', '_', regex=False)          # Replace dashes with underscores
    .str.replace('\n', '', regex=False)          # Remove newline characters
)

# Single replacement operation for the 'CDID' column
merged_df['CDID'] = merged_df['CDID'].str.replace(' ', '', regex=False)

# Display the first few rows of the transformed DataFrame
print(merged_df.head())
print("Complete")


   Year Quarter         NamedCost        Values COICOP  CDID Source  \
0  1997      Q1  Totalexpenditure  1.448870e+11   NAT0  ABPB  0GSCN   
1  1997      Q2  Totalexpenditure  1.504740e+11   NAT0  ABPB  0GSCN   
2  1997      Q3  Totalexpenditure  1.558090e+11   NAT0  ABPB  0GSCN   
3  1997      Q4  Totalexpenditure  1.591930e+11   NAT0  ABPB  0GSCN   
4  1998      Q1  Totalexpenditure  1.533470e+11   NAT0  ABPB  0GSCN   

                              Websource  
0  https://www.ons.gov.uk/search?q=ABPB  
1  https://www.ons.gov.uk/search?q=ABPB  
2  https://www.ons.gov.uk/search?q=ABPB  
3  https://www.ons.gov.uk/search?q=ABPB  
4  https://www.ons.gov.uk/search?q=ABPB  
Complete


In [5]:
# Step 1 & 2: Identify all 'CDID' values for 'Source' = '0GSCN' and check which of these 'CDID' values exist in rows with a different 'Source'
cdids_for_0GSCN = merged_df[merged_df['Source'] == '0GSCN']['CDID'].unique()
is_duplicated_cdids = merged_df['CDID'].isin(cdids_for_0GSCN) & (merged_df['Source'] != '0GSCN')
duplicated_cdids = merged_df[is_duplicated_cdids]['CDID'].unique()

# Step 3: Apply vectorized logic to create the 'Grouptotal' column
condition1 = merged_df['CDID'].isin(duplicated_cdids)
condition2 = (merged_df['COICOP'] == 'TD') & (merged_df['CDID'] == 'UTIA') & (merged_df['Source'] == '0GSCN')
merged_df['Grouptotal'] = (condition1 | condition2).astype(int)

# Display the first few rows of the updated DataFrame
print(merged_df.head())

# Save the cleaned DataFrame to a new CSV file
cleaned_file_path = 'C:/Users/l_cla/Documents/000_Project/000_000_datastored/200_structured/dataset.csv'
merged_df.to_csv(cleaned_file_path, index=False)
print("Complete")


   Year Quarter         NamedCost        Values COICOP  CDID Source  \
0  1997      Q1  Totalexpenditure  1.448870e+11   NAT0  ABPB  0GSCN   
1  1997      Q2  Totalexpenditure  1.504740e+11   NAT0  ABPB  0GSCN   
2  1997      Q3  Totalexpenditure  1.558090e+11   NAT0  ABPB  0GSCN   
3  1997      Q4  Totalexpenditure  1.591930e+11   NAT0  ABPB  0GSCN   
4  1998      Q1  Totalexpenditure  1.533470e+11   NAT0  ABPB  0GSCN   

                              Websource  Grouptotal  
0  https://www.ons.gov.uk/search?q=ABPB           1  
1  https://www.ons.gov.uk/search?q=ABPB           1  
2  https://www.ons.gov.uk/search?q=ABPB           1  
3  https://www.ons.gov.uk/search?q=ABPB           1  
4  https://www.ons.gov.uk/search?q=ABPB           1  
Complete


In [6]:
import sqlite3

# Create a new SQLite database
conn = sqlite3.connect("C:/Users/l_cla/Documents/000_Project/000_000_datastored/200_structured/dataset.db")
cursor = conn.cursor()

# Drop existing tables if they exist (for clean setup)
tables = ["FctExpenditure", "DimDate", "DimNamedCost", "DimIdentifier"]
for table in tables:
    cursor.execute(f"DROP TABLE IF EXISTS {table}")

# Create dimension tables

# DateDim table
cursor.execute('''
CREATE TABLE DimDate (
    DateID INTEGER PRIMARY KEY AUTOINCREMENT,
    Year INTEGER,
    Quarter TEXT
)
''')

# NamedCostDim table
cursor.execute('''
CREATE TABLE DimNamedCost (
    NamedCostID INTEGER PRIMARY KEY AUTOINCREMENT,
    NamedCost TEXT UNIQUE
)
''')

# IdentifierDim table
cursor.execute('''
CREATE TABLE DimIdentifier (
    IdentifierID INTEGER PRIMARY KEY AUTOINCREMENT,
    COICOP TEXT,
    CDID TEXT,
    Source TEXT,
    Websource TEXT
)
''')

# Create the fact table

# ExpenditureFacts table
cursor.execute('''
CREATE TABLE FctExpenditure (
    FactID INTEGER PRIMARY KEY AUTOINCREMENT,
    DateID INTEGER,
    NamedCostID INTEGER,
    IdentifierID INTEGER,
    ExpenditureValue REAL,
    GroupTotal INTEGER,
    FOREIGN KEY (DateID) REFERENCES DateDim(DateID),
    FOREIGN KEY (NamedCostID) REFERENCES NamedCostDim(NamedCostID),
    FOREIGN KEY (IdentifierID) REFERENCES IdentifierDim(IdentifierID)
)
''')

# Populate DateDim table
dates = merged_df[['Year', 'Quarter']].drop_duplicates().to_dict('records')
for date in dates:
    cursor.execute('INSERT INTO DimDate (Year, Quarter) VALUES (?, ?)', (date['Year'], date['Quarter']))

# Populate NamedCostDim table
named_costs = merged_df['NamedCost'].drop_duplicates().tolist()
for cost in named_costs:
    cursor.execute('INSERT INTO DimNamedCost (NamedCost) VALUES (?)', (cost,))

# Populate IdentifierDim table
identifiers = merged_df[['COICOP', 'CDID', 'Source', 'Websource']].drop_duplicates().to_dict('records')
for identifier in identifiers:
    cursor.execute('''
    INSERT INTO DimIdentifier (COICOP, CDID, Source, Websource)
    VALUES (?, ?, ?, ?)
    ''', (identifier['COICOP'], identifier['CDID'], identifier['Source'], identifier['Websource']))

# Populate ExpenditureFacts table
for index, row in merged_df.iterrows():
    date_id = cursor.execute('SELECT DateID FROM DimDate WHERE Year = ? AND Quarter = ?', (row['Year'], row['Quarter'])).fetchone()[0]
    named_cost_id = cursor.execute('SELECT NamedCostID FROM DimNamedCost WHERE NamedCost = ?', (row['NamedCost'],)).fetchone()[0]
    identifier_id = cursor.execute('''
    SELECT IdentifierID FROM DimIdentifier WHERE COICOP = ? AND CDID = ? AND Source = ? AND Websource = ?
    ''', (row['COICOP'], row['CDID'], row['Source'], row['Websource'])).fetchone()[0]
    
    cursor.execute('''
    INSERT INTO FctExpenditure (DateID, NamedCostID, IdentifierID, ExpenditureValue, GroupTotal)
    VALUES (?, ?, ?, ?, ?)
    ''', (date_id, named_cost_id, identifier_id, row['Values'], row['Grouptotal']))

# Commit the changes to the database
conn.commit()
conn.close()


In [7]:
import sqlite3
import pandas as pd

# Reconnect to the SQLite database
conn = sqlite3.connect("C:/Users/l_cla/Documents/000_Project/000_000_datastored/200_structured/dataset.db")
cursor = conn.cursor()

# Define the SQL query to fetch data and perform joins
query = '''
SELECT 
    DimDate.Year, DimDate.Quarter,
    DimNamedCost.NamedCost,
    DimIdentifier.COICOP, DimIdentifier.CDID, DimIdentifier.Source, DimIdentifier.Websource,
    FctExpenditure.ExpenditureValue, FctExpenditure.GroupTotal
FROM FctExpenditure
JOIN DimDate ON FctExpenditure.DateID = DimDate.DateID
JOIN DimNamedCost ON FctExpenditure.NamedCostID = DimNamedCost.NamedCostID
JOIN DimIdentifier ON FctExpenditure.IdentifierID = DimIdentifier.IdentifierID
'''

# Execute the query and fetch all records
records = cursor.execute(query).fetchall()

# Get the column names from the cursor's description
column_names = [desc[0] for desc in cursor.description]

# Convert records to a DataFrame
records_df = pd.DataFrame(records, columns=column_names)

# Convert the 'ExpenditureValue' column from scientific notation to float with two decimal places
records_df['ExpenditureValue'] = records_df['ExpenditureValue'].apply(lambda x: '{:,.2f}'.format(x))

# Display the first few records of the DataFrame (remove '.head()' to display all)
records_df.head()

,Year,Quarter,NamedCost,COICOP,CDID,Source,Websource,ExpenditureValue,GroupTotal
0,1997,Q1,Totalexpenditure,NAT0,ABPB,0GSCN,https://www.ons.gov.uk/search?q=ABPB,"144,887,000,000.00",1
1,1997,Q2,Totalexpenditure,NAT0,ABPB,0GSCN,https://www.ons.gov.uk/search?q=ABPB,"150,474,000,000.00",1
2,1997,Q3,Totalexpenditure,NAT0,ABPB,0GSCN,https://www.ons.gov.uk/search?q=ABPB,"155,809,000,000.00",1
3,1997,Q4,Totalexpenditure,NAT0,ABPB,0GSCN,https://www.ons.gov.uk/search?q=ABPB,"159,193,000,000.00",1
4,1998,Q1,Totalexpenditure,NAT0,ABPB,0GSCN,https://www.ons.gov.uk/search?q=ABPB,"153,347,000,000.00",1


(32240, 9)